In [5]:
import os

os.environ["OPENAI_API_TYPE"] = "azure"
os.environ["OPENAI_API_BASE"] = ""
os.environ["OPENAI_API_KEY"] = ""

# Appointment API

In [6]:
import yaml
from langchain.llms.openai import OpenAI
from langchain.agents.agent_toolkits.openapi import planner
from langchain.agents.agent_toolkits.openapi.spec import reduce_openapi_spec
from langchain.requests import RequestsWrapper

In [7]:
with open("letstalk.yml") as f:
    raw_lets_talk_spec = yaml.load(f, Loader=yaml.Loader)
lets_talk_spec = reduce_openapi_spec(raw_lets_talk_spec)

In [8]:
lets_talk_spec

ReducedOpenAPISpec(servers=[{'url': 'http://localhost:8081/'}], description='', endpoints=[('GET /', 'Default informative end-point running on root address', {'description': 'Default informative end-point running on root address', 'responses': {'content': {'application/json': {'schema': {}}}, 'description': 'Successful Response'}}), ('GET /api/appointment', 'List all items in the database', {'description': 'List all items in the database', 'responses': {'content': {'application/json': {'schema': {'items': {'properties': {'condition_id': {'title': 'Condition Id', 'type': 'integer'}, 'date_and_time': {'format': 'date-time', 'title': 'Date And Time', 'type': 'string'}, 'details': {'title': 'Details', 'type': 'string'}, 'id': {'title': 'Id', 'type': 'integer'}}, 'required': ['id', 'date_and_time', 'condition_id'], 'title': 'Appointment', 'type': 'object'}, 'title': 'Response List Item Api Appointment Get', 'type': 'array'}}}, 'description': 'Successful Response'}}), ('POST /api/appointment

In [9]:
requests_wrapper = RequestsWrapper(headers={})

In [10]:
llm = OpenAI(engine="text-davinci-003", temperature=0.0)

WARNING! engine is not default parameter.
                    engine was transferred to model_kwargs.
                    Please confirm that engine is what you intended.


In [11]:
letstalk_agent = planner.create_openapi_agent(lets_talk_spec, requests_wrapper, llm)


In [ ]:
letstalk_agent.run("list all available doctors")

In [ ]:
letstalk_agent.run("create a doctor named Menedoctor da Silva with specialty in cardiology")

# QA Agent

In [13]:
with open("qa_system.yml") as f:
    raw_qa_spec = yaml.load(f, Loader=yaml.Loader)
qa_spec = reduce_openapi_spec(raw_qa_spec)

In [24]:
qa_spec

ReducedOpenAPISpec(servers=[{'url': 'http://localhost:8000/'}], description='', endpoints=[('GET /', None, {'responses': {'content': {'application/json': {'schema': {}}}, 'description': 'Successful Response'}}), ('POST /ask', 'Gets a user query as input and use it to answer a question usar a knowledge base as reference', {'description': 'Gets a user query as input and use it to answer a question usar a knowledge base as reference', 'parameters': [{'in': 'query', 'name': 'query', 'required': True, 'schema': {'title': 'Query', 'type': 'string'}}], 'responses': {'content': {'application/json': {'schema': {}}}, 'description': 'Successful Response'}})])

In [14]:
qa_agent = planner.create_openapi_agent(qa_spec, requests_wrapper, llm)


# Agent

In [16]:
from langchain.agents import AgentType, initialize_agent
from langchain.chat_models import ChatOpenAI
from langchain.tools import BaseTool, StructuredTool, Tool, tool

In [17]:
llm = ChatOpenAI(engine='gpt-35-turbo', temperature=0)

WARNING! engine is not default parameter.
                    engine was transferred to model_kwargs.
                    Please confirm that engine is what you intended.


In [18]:
tools = [
    Tool.from_function(
        func=letstalk_agent.run,
        name = "Lets Talk API",
        description="useful for when you need to manipulate a database that has data about appointments with doctors"
    ),
    Tool.from_function(
        func=qa_agent.run,
        name = "QA System",
        description="useful for when a user has questions about a knowledge base"
    ),
]

In [19]:
agent = initialize_agent(tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)

In [23]:
agent.run("Eu preciso fazer alguma coisa pra receber vocês em casa?")



> Entering new AgentExecutor chain...
I'm not sure what the context is or what the person is asking for specifically.
Action: QA System
Action Input: "What should I do to prepare for guests coming over?"

> Entering new AgentExecutor chain...
Action: api_planner
Action Input: I need to find the right API calls to prepare for guests coming over
Observation:  1. GET /ask with a query param to search for information about preparing for guests
2. GET /user to find the user's id
3. POST /users/{id}/prepare to add the necessary items to the user's cart
4. PATCH /users/{id}/prepare to update the user's cart with any additional items needed
Thought: I'm ready to execute the API calls.
Action: api_controller
Action Input: 1. GET /ask with a query param to search for information about preparing for guests
2. GET /user to find the user's id
3. POST /users/{id}/prepare to add the necessary items to the user's cart
4. PATCH /users/{id}/prepare to update the user's cart with any additional items n

ValueError: GET /ask endpoint does not exist.